In [1]:
pip install pycocotools

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import tensorflow as tf
from pycocotools.coco import COCO

2023-05-14 16:41:56.232052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 16:41:57.379068: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-14 16:41:57.379105: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-14 16:42:00.525348: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
dataset_dir = '/home/dheeraj/Documents/sitting-posture-detection/dataset/train2017'
annotation_file = '/home/dheeraj/Documents/sitting-posture-detection/annotations/annotations_trainval2017/annotations/person_keypoints_train2017.json'

In [4]:
coco = COCO(annotation_file)

loading annotations into memory...
Done (t=13.95s)
creating index...
index created!


In [5]:
image_ids = coco.getImgIds()
category_ids = coco.getCatIds()

In [6]:
image_ids

[391895,
 522418,
 184613,
 318219,
 554625,
 574769,
 60623,
 309022,
 5802,
 222564,
 118113,
 193271,
 224736,
 483108,
 403013,
 374628,
 328757,
 384213,
 293802,
 86408,
 372938,
 386164,
 223648,
 204805,
 113588,
 384553,
 337264,
 368402,
 12448,
 79841,
 515289,
 562150,
 542145,
 412151,
 579003,
 540186,
 242611,
 51191,
 269105,
 294832,
 462565,
 144941,
 173350,
 60760,
 324266,
 166532,
 262284,
 360772,
 191381,
 111076,
 340559,
 258985,
 509822,
 321107,
 229643,
 125059,
 455483,
 436141,
 129001,
 232262,
 61181,
 166323,
 580041,
 326781,
 387362,
 138079,
 556616,
 472621,
 192440,
 86320,
 256668,
 383445,
 565797,
 81922,
 50125,
 364521,
 394892,
 1146,
 310391,
 97434,
 463836,
 241876,
 156832,
 270721,
 462341,
 310103,
 32992,
 122851,
 540763,
 138246,
 197254,
 32907,
 251252,
 37675,
 159537,
 268556,
 271177,
 75051,
 549399,
 85160,
 559665,
 19358,
 459912,
 15827,
 394240,
 510755,
 175831,
 79472,
 250108,
 315601,
 437218,
 503707,
 74331,
 579664

In [7]:
dataset = []
for image_id in image_ids:
    image_info = coco.loadImgs(image_id)[0]
    image_path = os.path.join(dataset_dir, image_info['file_name'])
    annotations_ids = coco.getAnnIds(imgIds=image_id, catIds=category_ids, iscrowd=None)
    annotations = coco.loadAnns(annotations_ids)
    dataset.append({
        'image_path': image_path,
        'annotations': annotations
    })

In [8]:
#dataset size 
print(len(dataset))

118287


In [9]:
print(dataset[0:10])

[{'image_path': '/home/dheeraj/Documents/sitting-posture-detection/dataset/train2017/000000391895.jpg', 'annotations': [{'segmentation': [[352.55, 146.82, 353.61, 137.66, 356.07, 112.66, 357.13, 94.7, 357.13, 84.49, 363.12, 73.92, 370.16, 68.64, 370.16, 66.53, 368.4, 63.71, 368.05, 54.56, 361, 53.85, 356.07, 50.33, 356.43, 46.46, 364.17, 42.23, 369.1, 35.89, 371.22, 30.96, 376.85, 26.39, 383.54, 22.16, 391.29, 23.22, 400.79, 27.79, 402.2, 30.61, 404.32, 34.84, 406.08, 38.71, 406.08, 41.53, 406.08, 47.87, 407.84, 54.91, 408.89, 59.84, 408.89, 61.25, 408.89, 63.36, 422.28, 67.94, 432.13, 72.52, 445.87, 81.32, 446.57, 84.14, 446.57, 99.2, 451.15, 118.22, 453.26, 128.39, 453.61, 131.92, 453.61, 133.68, 451.5, 137.55, 451.5, 139.31, 455.38, 144.24, 455.38, 153.04, 455.73, 155.16, 461.01, 162.85, 462.07, 166.37, 459.95, 170.6, 459.6, 176.58, 459.95, 178.69, 459.95, 180.1, 448.33, 180.45, 447.98, 177.64, 446.57, 172.36, 447.63, 166.37, 449.74, 160.38, 450.09, 157.57, 448.68, 152.28, 445.16, 1

In [10]:
import cv2
import numpy as np
import multiprocessing
import random

Now we are doing preprocessing of dataset


In [11]:
def resize_image(image, target_size):
    resized_image = cv2.resize(image, target_size)
    return resized_image

In [12]:
def normalize_image(image):
    normalized_image = image / 255.0
    return normalized_image

In [13]:
# Keypoint normalization
def normalize_keypoints(keypoints, image_width, image_height):
    normalized_keypoints = keypoints.copy()
    normalized_keypoints[::3] = [kp / image_width for kp in normalized_keypoints[::3]]
    normalized_keypoints[1::3] = [kp / image_height for kp in normalized_keypoints[1::3]]
    return normalized_keypoints

In [14]:
# Data augmentation (example: random horizontal flipping)
def augment_image(image, keypoints):
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        flipped_keypoints = keypoints.copy()
        flipped_keypoints[::3] = [1 - kp for kp in flipped_keypoints[::3]]
        return image, flipped_keypoints
    return image, keypoints

In [15]:
def compute_vec_weight_input(image, keypoints, bbox):
    num_keypoints = keypoints.shape[0]
    vec_weight_input = np.ones((num_keypoints, 1))
    return vec_weight_input

In [16]:
def compute_heat_weight_input(image, keypoints, bbox):
    center_x = (bbox[0] + bbox[2]) / 2
    center_y = (bbox[1] + bbox[3]) / 2

    if len(keypoints.shape) == 2:
        distances = np.sqrt(np.square(keypoints[:, 0] - center_x) + np.square(keypoints[:, 1] - center_y))
    else:
        distances = np.sqrt(np.square(keypoints[0] - center_x) + np.square(keypoints[1] - center_y))

    max_distance = np.max(distances)
    heat_weight_input = np.exp(-(distances / max_distance))
    
    return heat_weight_input

In [17]:
def preprocess_batch(image_paths, annotations, target_size=(224, 224)):
    preprocessed_batch = []
    for i in range(len(image_paths)):
        image_path = image_paths[i]
        annotation = annotations[i]
        
        # Load and preprocess the image
        image = cv2.imread(image_path)
        image = resize_image(image, target_size)
        image = normalize_image(image)
        
        # Process annotations
        for annotation_item in annotation:
            keypoints = annotation_item['keypoints']
            bbox = annotation_item['bbox']
            
            # Normalize keypoints
            image_height, image_width = image.shape[:2]
            normalized_keypoints = normalize_keypoints(keypoints, image_width, image_height)
            
            # Augmentation (optional)
            image_aug, keypoints_aug = augment_image(image, normalized_keypoints)
            
            # Convert keypoints_aug to numpy array
            keypoints_aug = np.array(keypoints_aug)
            
            # Compute vec_weight_input and heat_weight_input
            vec_weight_input = compute_vec_weight_input(image_aug, keypoints_aug, bbox)
            heat_weight_input = compute_heat_weight_input(image_aug, keypoints_aug, bbox)
            
            # Add preprocessed data to the batch
            preprocessed_batch.append({
                'image': image_aug,
                'keypoints': keypoints_aug,
                'bbox': bbox,
                'vec_weight_input': vec_weight_input,
                'heat_weight_input': heat_weight_input
            })
    
    return preprocessed_batch

In [18]:
# def preprocess_data(data):
#     image_paths = [d['image_path'] for d in data]
#     annotations = [d['annotations'] for d in data]
#     preprocessed_batch = preprocess_batch(image_paths, annotations)
#     return preprocessed_batch

In [19]:
# preprocessed_dataset = []
# batch_size = 100
# num_processes = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=num_processes)

# for i in range(0, 10000, batch_size):
#     batch = dataset[i:i+batch_size]
#     results = pool.map(preprocess_data, [batch])
#     preprocessed_dataset.extend(results[0])

# pool.close()

In [20]:
# def data_generator(dataset, batch_size):
#     for i in range(0, 5000, batch_size):
#         batch = dataset[i:i+batch_size]
#         image_paths = [data['image_path'] for data in batch]
#         annotations = [data['annotations'] for data in batch]
#         preprocessed_batch = preprocess_batch(image_paths, annotations)
#         yield preprocessed_batch

In [21]:
# preprocessed_dataset = []
# batch_size = 50
# generator = data_generator(dataset, batch_size)

# for preprocessed_batch in generator:
#     preprocessed_dataset.extend(preprocessed_batch)

In [22]:
subset_size = 2500  # Define the desired subset size
small_dataset = random.sample(dataset, subset_size)

In [23]:
print(len(small_dataset))

2500


In [24]:
# Process the dataset in batches
preprocessed_dataset = []
batch_size = 1000
for i in range(0,1000, batch_size):
    # Extract a batch of data
    batch = small_dataset[i:i+batch_size]
    image_paths = [data['image_path'] for data in batch]
    annotations = [data['annotations'] for data in batch]
    
    # Preprocess the batch
    preprocessed_batch = preprocess_batch(image_paths, annotations)
    
    # Add preprocessed batch to the dataset
    preprocessed_dataset.extend(preprocessed_batch)

In [25]:
#preprocess_dataset size 
print(len(preprocessed_dataset))

2142


In [26]:
print(preprocessed_dataset[0])

{'image': array([[[0.10588235, 0.32156863, 0.45882353],
        [0.11764706, 0.31372549, 0.46666667],
        [0.13333333, 0.32941176, 0.48627451],
        ...,
        [0.03921569, 0.3372549 , 0.43921569],
        [0.05490196, 0.30196078, 0.4       ],
        [0.05098039, 0.30196078, 0.42745098]],

       [[0.1254902 , 0.29411765, 0.45882353],
        [0.11372549, 0.30980392, 0.47058824],
        [0.1254902 , 0.31764706, 0.46666667],
        ...,
        [0.04313725, 0.33333333, 0.43529412],
        [0.07843137, 0.30980392, 0.4       ],
        [0.07843137, 0.29411765, 0.39607843]],

       [[0.05490196, 0.29411765, 0.45882353],
        [0.09411765, 0.29411765, 0.45490196],
        [0.07843137, 0.29803922, 0.45882353],
        ...,
        [0.0745098 , 0.33333333, 0.44313725],
        [0.11372549, 0.30588235, 0.4       ],
        [0.09411765, 0.28627451, 0.38823529]],

       ...,

       [[0.01568627, 0.17647059, 0.15686275],
        [0.02745098, 0.16470588, 0.15686275],
        [0.0

In [27]:
# small_dataset = dataset[0:1000]

In [28]:
# # Preprocessing the dataset
# preprocessed_dataset = []
# batch_size = 100
# for data in small_dataset:
#     image_path = data['image_path']
#     annotations = data['annotations']
    
#     # Load and preprocess the image
#     image = cv2.imread(image_path)
#     image = resize_image(image, (224, 224))  # Resize to a fixed size
#     image = normalize_image(image)  # Normalize pixel values
    
#     # Process annotations
#     for annotation in annotations:
#         keypoints = annotation['keypoints']
#         bbox = annotation['bbox']
        
#         # Normalize keypoints
#         image_height, image_width = image.shape[:2]
#         normalized_keypoints = normalize_keypoints(keypoints, image_width, image_height)
        
#         # Augmentation (optional)
#         #image, normalized_keypoints = augment_image(image, normalized_keypoints)
        
#         # Add preprocessed data to the dataset
#         preprocessed_dataset.append({
#             'image': image,
#             'keypoints': normalized_keypoints,
#             'bbox': bbox
#         })


In [29]:
print(preprocessed_dataset[0])

{'image': array([[[0.10588235, 0.32156863, 0.45882353],
        [0.11764706, 0.31372549, 0.46666667],
        [0.13333333, 0.32941176, 0.48627451],
        ...,
        [0.03921569, 0.3372549 , 0.43921569],
        [0.05490196, 0.30196078, 0.4       ],
        [0.05098039, 0.30196078, 0.42745098]],

       [[0.1254902 , 0.29411765, 0.45882353],
        [0.11372549, 0.30980392, 0.47058824],
        [0.1254902 , 0.31764706, 0.46666667],
        ...,
        [0.04313725, 0.33333333, 0.43529412],
        [0.07843137, 0.30980392, 0.4       ],
        [0.07843137, 0.29411765, 0.39607843]],

       [[0.05490196, 0.29411765, 0.45882353],
        [0.09411765, 0.29411765, 0.45490196],
        [0.07843137, 0.29803922, 0.45882353],
        ...,
        [0.0745098 , 0.33333333, 0.44313725],
        [0.11372549, 0.30588235, 0.4       ],
        [0.09411765, 0.28627451, 0.38823529]],

       ...,

       [[0.01568627, 0.17647059, 0.15686275],
        [0.02745098, 0.16470588, 0.15686275],
        [0.0

In [30]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint

from model import get_training_model

In [31]:
loss_function = MeanSquaredError()

# Compile and train the model

In [43]:
model = get_training_model(weight_decay=0.001)

In [44]:
optimizer = Adam(learning_rate=0.001)

In [45]:
model.compile(optimizer = optimizer, loss='mean_squared_error')

In [46]:
# Define paths for saving checkpoints and logs
checkpoint_path = '/home/dheeraj/Documents/sitting-posture-detection/checkpoints/model_checkpoint.h5'
log_dir = '/home/dheeraj/Documents/sitting-posture-detection/logs'

In [47]:
# Define callbacks for saving checkpoints and tensorboard logs
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [48]:
#default_value = np.array([])

In [49]:
def data_generator(dataset, batch_size, default_value=0.0):
    num_samples = len(dataset)
    while True:
        indices = np.random.choice(num_samples, size=batch_size, replace=False)
        batch_images = []
        batch_keypoints = []
        batch_bboxes = []
        batch_vec_weight_input = []
        batch_heat_weight_input = []
        for idx in indices:
            data = dataset[idx]
            batch_images.append(data['image'])
            batch_keypoints.append(data['keypoints'])
            batch_bboxes.append(data['bbox'])
            batch_vec_weight_input.append(data['vec_weight_input'])
            batch_heat_weight_input.append(data['heat_weight_input'])
        batch_images = np.array(batch_images)
        batch_keypoints = np.array(batch_keypoints)
        batch_bboxes = np.array(batch_bboxes)
        batch_vec_weight_input = np.array(batch_vec_weight_input)
        batch_heat_weight_input = np.array(batch_heat_weight_input)

        # Replace None values with default value
        batch_images = np.array([default_value if image is None else image for image in batch_images])
        batch_keypoints = np.array([default_value if keypoints is None else keypoints for keypoints in batch_keypoints])
        batch_bboxes = np.array([default_value if bbox is None else bbox for bbox in batch_bboxes])
        batch_vec_weight_input = np.array([default_value if vec_weight_input is None else vec_weight_input for vec_weight_input in batch_vec_weight_input])
        batch_heat_weight_input = np.array([default_value if heat_weight_input is None else heat_weight_input for heat_weight_input in batch_heat_weight_input])

        yield [batch_images,batch_keypoints,batch_bboxes],[batch_keypoints,batch_bboxes]

In [50]:
# Create a generator instance
batch_size = 50
data_gen = data_generator(preprocessed_dataset, batch_size)

In [51]:
# Specify training parameters
epochs = 10
#batch_size = 32

In [52]:
# Start training
model.fit(
    data_gen,
    steps_per_epoch=32,
    epochs=epochs,
    callbacks=[checkpoint_callback, tensorboard_callback],
    verbose=1,
    y=None
#     validation_data=data_gen,
#     validation_steps=8
)

2023-05-14 16:46:30.444688: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 30105600 exceeds 10% of free system memory.


Epoch 1/10


ValueError: in user code:

    File "/home/dheeraj/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/dheeraj/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/dheeraj/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/dheeraj/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1028, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/dheeraj/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1122, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/dheeraj/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 577